# Import & Load NENE data

### Config

In [ ]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [ ]:
remote_path = 'http://pacha.datawheel.us/datachile/economy/nene/analysis/2_occupational_status/4_csv/'
local_path = '../data/'


### Imports

In [ ]:
import postgres #from local file postgres.py
from commons import inline_table_xml, download_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load file

In [ ]:
df = download_file(remote_path,local_path,'occupational_status.csv')

In [ ]:
df = df.astype({'year':'int','region_id':'int','age':'int','age_range_id':'int','icse_id':'int','sex_id':'int','isced_id':'int','isco_id':'int','occupied_id':'int','icse_id':'int','general_economic_condition':'int'})
list(df)

### Ingest

In [ ]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'economy', 'fact_occupational_status_nene_mobile_quarters')

### Indexes & FK

In [ ]:
engine.execute("""
UPDATE economy.fact_occupational_status_nene_mobile_quarters
SET mobile_quarter = concat(year,'_',mobile_quarter)
""")

engine.execute("""
CREATE INDEX fact_occupational_status_nene_mobile_quarters_region_id 
ON economy.fact_occupational_status_nene_mobile_quarters (region_id)
""")

### Related dim

In [ ]:
d1 = download_file(remote_path,local_path,'age_range_id.csv')
print (inline_table_xml(d1, 'age_range', 'age_range_id', 'age_range'))

In [ ]:
d2 = download_file(remote_path,local_path,'icse_id.csv')
print (inline_table_xml(d2, 'icse', 'icse_id', 'icse'))

In [ ]:
d3 = download_file(remote_path,local_path,'isced_id.csv')
print (inline_table_xml(d3, 'isced', 'isced_id', 'isced'))

In [ ]:
d4 = download_file(remote_path,local_path,'isco_id.csv')
print (inline_table_xml(d4, 'isco', 'isco_id', 'isco'))

In [ ]:
d5 = download_file(remote_path,local_path,'sex_id.csv')
print (inline_table_xml(d5, 'sex', 'sex_id', 'sex'))

In [ ]:
d6 = download_file(remote_path,local_path,'general_economic_condition_id.csv')
print (inline_table_xml(d6, 'general_economic_condition', 'general_economic_condition_id', 'general_economic_condition'))

In [ ]:
d7 = download_file(remote_path,local_path,'occupied_id.csv')
print (inline_table_xml(d7, 'occupied', 'occupied_id', 'occupied'))